# Lab 8: CellRank 2 — VelocityKernel Fate Mapping

**Module 8** — RNA Velocity-Based Fate Mapping with CellRank 2

## Objectives
- Build a transition matrix from RNA velocity using CellRank 2's VelocityKernel
- Combine VelocityKernel with ConnectivityKernel
- Identify terminal states via GPCCA macrostate analysis
- Compute fate probabilities and find driver genes
- Visualize gene expression trends along lineages

## Reference
- Weiler & Theis (2026) *Nature Protocols* [doi:10.1038/s41596-025-01314-w](https://doi.org/10.1038/s41596-025-01314-w)
- CellRank Protocol: [github.com/theislab/cellrank_protocol](https://github.com/theislab/cellrank_protocol) — `notebooks/velocity/`

---

## 1. Setup & Data Loading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import scvelo as scv
import cellrank as cr

scv.settings.verbosity = 3
scv.settings.set_figure_params(dpi=100, facecolor='white')
cr.settings.verbosity = 2

print(f"scanpy:   {sc.__version__}")
print(f"scvelo:   {scv.__version__}")
print(f"cellrank: {cr.__version__}")

In [ ]:
# Load the pancreas endocrine differentiation dataset
# This is a classic trajectory dataset with spliced/unspliced counts
adata = scv.datasets.pancreas()
print(f"Cells: {adata.n_obs}, Genes: {adata.n_vars}")
print(f"Layers: {list(adata.layers.keys())}")
print(f"Cell types: {adata.obs['clusters'].unique().tolist()}")

## 2. Preprocessing & RNA Velocity

Before using CellRank's VelocityKernel, we need RNA velocity estimates from scVelo.

In [ ]:
# Standard scVelo preprocessing
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

# Compute velocity using the dynamical model (recommended)
scv.tl.recover_dynamics(adata, n_jobs=4)
scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)

print("RNA velocity computed (dynamical model)")

In [ ]:
# Visualize velocity stream plot
scv.pl.velocity_embedding_stream(
    adata, basis='umap', color='clusters',
    title='RNA Velocity Stream Plot',
    figsize=(8, 6)
)

## 3. CellRank 2: VelocityKernel

The VelocityKernel translates RNA velocity vectors into a cell-cell **transition matrix**.
Each entry T[i,j] represents the probability that cell i transitions toward cell j.

We combine it with a ConnectivityKernel (based on transcriptomic similarity)
to smooth out noise in velocity estimates.

In [ ]:
from cellrank.kernels import VelocityKernel, ConnectivityKernel

# Step 1: Create the VelocityKernel
vk = VelocityKernel(adata)
vk.compute_transition_matrix()
print(f"VelocityKernel transition matrix: {vk.transition_matrix.shape}")

# Step 2: Create the ConnectivityKernel (transcriptomic similarity)
ck = ConnectivityKernel(adata)
ck.compute_transition_matrix()
print(f"ConnectivityKernel transition matrix: {ck.transition_matrix.shape}")

In [ ]:
# Step 3: Combine kernels with weights
# 80% velocity signal + 20% connectivity smoothing
combined_kernel = 0.8 * vk + 0.2 * ck
print(f"Combined kernel: {combined_kernel}")

# EXERCISE: Try different weight combinations and observe the effect
# What happens with 0.5 * vk + 0.5 * ck? Or 1.0 * vk?

## 4. GPCCA: Macrostate Identification

CellRank uses **Generalized Perron Cluster Cluster Analysis (GPCCA)** to identify
macrostates — groups of cells that the Markov chain tends to stay in.

The Schur decomposition reveals the eigenvalue spectrum; the number of real
eigenvalues close to 1 indicates the number of metastable macrostates.

In [ ]:
from cellrank.estimators import GPCCA

# Create GPCCA estimator from the combined kernel
g = GPCCA(combined_kernel)

# Compute Schur decomposition — look at eigenvalue spectrum
g.compute_schur(n_components=20)
g.plot_spectrum(real_only=True)
print("\nLook for a gap in the eigenvalue spectrum to choose n_states.")

In [ ]:
# Compute macrostates
# Choose n_states based on the eigenvalue gap observed above
g.compute_macrostates(n_states=6, cluster_key='clusters')

# Visualize macrostates on UMAP
g.plot_macrostates(which='all', basis='umap', legend_loc='right', s=30)

# Print macrostate composition
print("\nMacrostates identified:")
print(g.macrostates.cat.categories.tolist())

In [ ]:
# Set terminal states from the macrostates
# CellRank automatically identifies which macrostates are terminal (absorbing)
g.set_terminal_states()

# Visualize terminal states
g.plot_macrostates(which='terminal', basis='umap', legend_loc='right', s=30)

print("\nTerminal states:")
print(g.terminal_states.cat.categories.tolist())

## 5. Fate Probabilities

Given the terminal states, CellRank computes the **absorption probability** for each cell
to reach each terminal state. These are the **fate probabilities** — for every cell,
they sum to 1 across all terminal states.

In [ ]:
# Compute fate probabilities
g.compute_fate_probabilities()

# Visualize fate probabilities for each terminal state
g.plot_fate_probabilities(basis='umap', ncols=3, figsize=(15, 5))

# The fate probabilities are stored in adata.obsm['lineages_fwd']
print(f"\nFate probability matrix shape: {g.fate_probabilities.shape}")
print(f"Columns (lineages): {g.fate_probabilities.names.tolist()}")

In [ ]:
# Circular projection plot — shows fate bias for each cell type
g.plot_fate_probabilities(
    mode='violin',
    cluster_key='clusters',
    figsize=(12, 4)
)

## 6. Driver Genes

CellRank identifies **driver genes** — genes whose expression correlates with
the fate probability toward a specific terminal state. These are candidate
regulators of lineage commitment.

In [ ]:
# Compute driver genes for each lineage
# Uses correlation between gene expression and fate probabilities
terminal_states = g.terminal_states.cat.categories.tolist()

for lineage in terminal_states:
    drivers = g.compute_lineage_drivers(lineages=lineage, return_drivers=True)
    print(f"\nTop 10 driver genes for {lineage}:")
    print(drivers.head(10)[[f"{lineage}_corr", f"{lineage}_pval"]].to_string())

## 7. Gene Expression Trends Along Lineages

Visualize how gene expression changes along the fate probability gradient
toward each terminal state.

In [ ]:
# Select key genes (known markers or top drivers)
# Adjust gene names based on your dataset
genes_of_interest = ['Ins1', 'Gcg', 'Sst', 'Ghrl', 'Pax6', 'Neurog3']

# Filter to genes present in the dataset
genes_present = [g_name for g_name in genes_of_interest if g_name in adata.var_names]

if genes_present:
    model = cr.models.GAM(adata)
    cr.pl.gene_trends(
        adata,
        model=model,
        genes=genes_present[:4],
        data_key='X',
        figsize=(12, 8),
        ncols=2
    )
else:
    print("Adjust gene names to match your dataset")

## 8. Exercises

### Exercise 8.1: Kernel Weight Sensitivity
Rerun the analysis with different VelocityKernel / ConnectivityKernel weight ratios:
- `1.0 * vk` (velocity only)
- `0.5 * vk + 0.5 * ck`
- `0.2 * vk + 0.8 * ck`

How do the terminal states and fate probabilities change?

### Exercise 8.2: Number of Macrostates
Try `compute_macrostates(n_states=3)` and `n_states=8`.
Which seems most biologically appropriate? Why?

### Exercise 8.3: Driver Gene Validation
For your top 5 driver genes per lineage:
1. Are they known markers of those cell types?
2. Search them in a gene database (e.g., GeneCards)
3. Would you trust these as regulatory candidates?

### Exercise 8.4: Compare with Lab 6 Velocity
Compare the directionality from the velocity stream plot (Lab 6) with
CellRank's fate probabilities. Do they agree? Where do they disagree?

---

## Summary

In this lab you learned to:
- Build a VelocityKernel and ConnectivityKernel in CellRank 2
- Combine kernels with weighted addition
- Use GPCCA to identify macrostates and terminal states
- Compute fate probabilities (absorption probabilities)
- Find driver genes for each lineage
- Visualize gene trends along lineage fate probabilities

**Next:** Lab 9A explores the CytoTRACEKernel for velocity-free fate mapping.